# 1. Import Libraries and Load Data

In [ ]:
from transformers import AutoTokenizer, AutoModel
#https://huggingface.co/monsoon-nlp/bert-base-thai
pretrained  = 'monsoon-nlp/bert-base-thai'

In [ ]:
name = pretrained

In [ ]:
!pip -q install simpletransformers

In [ ]:
!pip -q install pythainlp

In [ ]:
#https://huggingface.co/datasets/Wongnai/wongnai_reviews
from datasets import load_dataset
wongnai_ds = load_dataset('wongnai_reviews')

In [ ]:
wongnai_ds

In [ ]:
import numpy as np
import pandas as pd
num_labels = len(np.unique(np.array(np.array(wongnai_ds['train']['star_rating']))))

In [ ]:
num_labels

In [ ]:
wongnai = pd.DataFrame(wongnai_ds['train'])

In [ ]:
#Use small dataset
wongnai_train = wongnai.sample(frac=0.5) #0.2
wongnai_eval = wongnai.sample(frac=0.1) #0.04
print('wongnai_train',wongnai_train.shape)
print('wongnai_eval ',wongnai_eval .shape)

In [ ]:
wongnai_eval

In [ ]:
#https://simpletransformers.ai/docs/classification-data-formats/
!pip install matplotlib
from pythainlp.tokenize import word_tokenize
wongnai_train['text'] = wongnai_train['review_body'].apply(lambda x:' '.join(word_tokenize(x)))
wongnai_train['labels'] = wongnai_train['star_rating']
wongnai_train.labels = pd.Categorical(wongnai_train.labels)

In [ ]:
wongnai_eval['text'] = wongnai_eval['review_body'].apply(lambda x:' '.join(word_tokenize(x)))
wongnai_eval['labels'] = wongnai_eval['star_rating']
wongnai_eval.labels = pd.Categorical(wongnai_eval.labels)

In [ ]:
wongnai_train

In [ ]:
wongnai_train['labels'].hist()

In [ ]:
wongnai_train = pd.concat([
    wongnai_train[wongnai_train['labels']==0].sample(n=500,replace = True),
    wongnai_train[wongnai_train['labels']==1].sample(n=500,replace = True),
    wongnai_train[wongnai_train['labels']==2].sample(n=500,replace = True),
    wongnai_train[wongnai_train['labels']==3].sample(n=500,replace = True),
    wongnai_train[wongnai_train['labels']==4].sample(n=500,replace = True),

],axis = 0)

In [ ]:
wongnai_train['labels'].hist()

# 3. Model Training and Evaluation

In [ ]:
!pip install torch torchvision torchaudio

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn
model_args = {
    'train_batch_size':32,
    'num_train_epochs': 150,
    'use_cached_eval_features': False,
    'no_cache': True,
    'overwrite_output_dir': True
    #'early_stopping_patience': 3,  # Number of epochs with no improvement before stopping
    #'early_stopping_delta': 0.01,  # Minimum change in evaluation metric to be considered an improvement
    #'evaluate_during_training': True,  # Required for early stopping
}

In [ ]:
model = ClassificationModel(
    'bert',
    name,
    args = model_args,
    num_labels=num_labels,
)

history = model.train_model(wongnai_train, eval_df = wongnai_eval, acc = sklearn.metrics.accuracy_score)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(wongnai_eval)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

y_true = wongnai_eval['labels'].values
y_pred = np.argmax(model_outputs, axis=1)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=range(num_labels), yticklabels=range(num_labels))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# classification report
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


In [ ]:
model.predict(['แย่ มาก กิน ไม่ ได้ เลย'])